**ANUSHKA GUPTA**
--
**NIT PATNA**
--
**PROBLEM STATEMENT 1**
--

* **DATA AUGEMENTATION**
    1. Back translation
    2. Synonym Swipe
    3. Random deletion
    4. Random Insertion

* **DATA CLEANING**
    1. Removing extra spaces
    2. Removing punctutations
    3. Case normalization
    4. Remvoing stop words
    5. Stemming
    6. Lemmetization

* **CREATION of CORPUS**
* **CREATION OF TF-IDF vector**
* **ENCODING OF CATEGORIES**
* **MODELLING and TRAINING**
* **TESTING AND RESULT**
* **PERFORMANCE ANALYSIS(Precision, Recall, Accuracy, F1 score)**
* **DATA VISUALIZATION** 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import random
!pip install googletrans==4.0.0-rc1

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns

import string
from wordcloud import WordCloud

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**IMPORTING TRAIN DATA**
------------------------

In [2]:
train_data = pd.read_csv("../input/instabase/data_csv.csv") 
train_data.head()

**IMPORTING TEST DATA**
-----------------------

In [3]:
test_data = pd.read_csv('../input/test-data/test_data_csv.csv')
test_data.tail()

**DATA AUGMENTATION**
---------------------

****BACK TRANSLATION****

In [4]:
from googletrans import Translator
t = Translator()
for i in range(26):
    
    sentence = train_data['TEXT'][i]
    to_lang = t.translate(text = sentence, dest = 'hi').text
    back_to_eng = t.translate(text = to_lang, dest = 'en').text
    train_data.loc[len(train_data.index)] = [back_to_eng, train_data['TAGS'][i]] 

****SYNONYM REPLACEMENT****

In [5]:
from nltk.corpus import wordnet

def get_synonyms(word):
    
    synonyms = set()
    
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [6]:
from nltk.corpus import stopwords
stop_words = []
for w in stopwords.words('english'):
    stop_words.append(w)

In [7]:
def synonym_replacement(words, n):
    
    words = words.split()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= n: #only replace up to n words
            break

    sentence = ' '.join(new_words)

    return sentence

In [8]:
length = len(train_data)

for i in range(length):
    replacement = synonym_replacement(train_data['TEXT'][i], len(train_data['TEXT'][i].split())/2)
    #print(replacement)
    train_data.loc[len(train_data.index)] = [replacement, train_data['TAGS'][i]] 
    
train_data.tail()

****RANDOM DELETION****

In [9]:
def random_deletion(words, p):

    words = words.split()
    
    #obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    #randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    #if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    sentence = ' '.join(new_words)
    
    return sentence

In [10]:
length = len(train_data)

for i in range(length):
    replacement = random_deletion(train_data['TEXT'][i],0.25)
    #print(replacement)
    train_data.loc[len(train_data.index)] = [replacement, train_data['TAGS'][i]] 
    
train_data.tail()

**RANDOM INSERTION**

In [11]:
def random_insertion(words, n):
    
    words = words.split()
    new_words = words.copy()
    
    for _ in range(n):
        add_word(new_words)
        
    sentence = ' '.join(new_words)
    return sentence

def add_word(new_words):
    
    synonyms = []
    counter = 0
    
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
        
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

In [12]:
length = len(train_data)

for i in range(length):
    replacement = random_insertion(train_data['TEXT'][i],2)
    #print(replacement)
    train_data.loc[len(train_data.index)] = [replacement, train_data['TAGS'][i]] 
    
train_data.tail()

**DATA CLEANING**
------------------

**REMOVING EXTRA SPACES**

In [13]:
def remove_spaces(text):
    text = ' '.join(filter(None,text.split(' ')))
    return text

train_data['REMOVE_SPACES'] = train_data['TEXT'].apply(lambda x: remove_spaces(x))

**REMOVING PUNCTUATION AND CASE NORMALIZATION**
---

**CREATION OF CORPUS(BAG OF WORDS)**
----------------------

In [14]:
ps = PorterStemmer()
wordnet=WordNetLemmatizer()
first = train_data['TEXT'].tolist()
second = test_data['TEXT'].tolist()
first.extend(second)

corpus = []
for i in range(len(first)):
    review = re.sub('[^a-zA-Z]', ' ', first[i])
    review = review.lower()
    review = review.split()
    review = [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

**CREATION OF TF-IDF VECTOR**
-----------------------------
**Term Frequency — Inverse Document Frequency**

In [15]:
tfidf_v=TfidfVectorizer(max_features=2500,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus).toarray()

**ENCODING CATEGORIES INTO LABEL FOR BOTH TRAINING AND TEST DATA**
-------------------------------------------------------------------

In [16]:
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(train_data['TAGS'])
y_test = label_encoder.fit_transform(test_data['TAGS'])

**MODELLING**
-------------

**BUILDING RANDOM FOREST CLASSIFIER**

In [17]:
X_train = X[:416]
y_train = labels
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 300, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

**TESTING THE DATA**

In [18]:
X_test = X[416:]
y_pred_pa = classifier.predict(X_test)
print(y_pred_pa)

**CONFUSION MATRIX**

In [19]:
cm_pa = confusion_matrix(y_test, y_pred_pa) 
cm_pa.view()

**PRINTING THE RESULT**
-----------------------

In [20]:
label_cat = {0:'Governing Law', 1:'Indemnification', 2:'Lender Defaulting', 3:'Other', 4: 'Preamble'}
list = []
for label in y_pred_pa:
    list.append(label_cat[label])
        
result = pd.DataFrame( list, test_data['TEXT'])
result

**RECALL,PRECISION,F1 SCORE AND ACCURACY**
---

In [21]:
print(metrics.classification_report(y_test, y_pred_pa, digits=3))

------------------------------------------------------------------------------
**INORDER TO RE-RUN THE PROGRAM, PLEASE RESTART THE KERNAL AND THEN RUN ALL**
------------------------------------------------------------------------------

**DATA VISUALIZATION**
---

In [22]:
plt.figure(figsize=(10,10))
ax = sns.countplot(x= train_data['TAGS'],  data= train_data, order = train_data['TAGS'].value_counts().index )
for p, label in zip(ax.patches, train_data['TAGS'].value_counts()):   
    ax.annotate(label, (p.get_x()+0.25, p.get_height()+0.5))

**WORD CLOUD FOR RAW DATA**

In [23]:
wordcloud = WordCloud(background_color="white",width=800, height=500).generate(' '.join(train_data['TEXT'].tolist()))
plt.figure( figsize=(20,10), facecolor='k')
plt.imshow(wordcloud)

**WORD CLOUD FOR EACH CLASS**
1. Preamble
2. Indemnification
3. Governing Law
4. Other
5. Lender Defaulting

In [24]:
labels = ['Preamble', 'Indemnification', 'Governing Law', 'Other', 'Lender Defaulting']
length = len(train_data)
for label in labels:
    list = []
    for i in range(length):
        if(train_data['TAGS'][i] == label):
            list.append(train_data['TEXT'][i])
    
    wordcloud = WordCloud(background_color="white",width=500, height=500).generate(' '.join(list))
    plt.figure( figsize=(20,10), facecolor='k')
    plt.imshow(wordcloud)        

In [25]:
result.to_csv('result.csv', index=False, encoding='utf-8')

**HYPER-PARAMETER TUNING**
---

**PARAMS IN RANDOM FOREST CURRENTLY IN USE**

In [26]:
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(classifier.get_params())

**RANDOM HYPERPARAMETER GRID**

In [27]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

**HYPERPARAMETER TUNING USING RANDOM SEARCH**

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
param_grid = rf_random.best_params_
param_grid

**GRIDE SEARCH WITH CROSS VALIDATION**

In [ ]:
# from sklearn.model_selection import GridSearchCV
# rf = RandomForestClassifier()
# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

# # not working :(